# Check for Duplicates

### Reference Files

In [22]:
import pickle
current_UID_position = pickle.load(open('current_uid.pickle','r'))
print(current_UID_position)


37871


In [21]:
with open('current_uid.pickle','w') as q:
    pickle.dump(37871, q)

### Connect to Database

In [2]:
from pymongo import MongoClient
# Connect to the mongo local database
connection = MongoClient()
db = connection.bsa_files
collection = db.bsa_files

In [4]:
def convertPyPDFDate(origDate):
    if len(origDate) == 23:
        year = origDate[2:6]
        mon = origDate[6:8]
        day = origDate[8:10]
        return (year+"-"+mon+"-"+day)
    return origDate

def download_from_url(url):
    import urllib2
    import PyPDF2
    from StringIO import StringIO
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    
    try: 
        tempReq = urllib2.Request(url, headers=hdr) 
        try: 
            tempPage = urllib2.urlopen(tempReq)
            print("page opened")
        except urllib2.HTTPError, e: 
            print(e.fp.read())
        
        tempPdf = PyPDF2.PdfFileReader(StringIO(tempPage.read()))
        print("temporary PDF file created")
        
        if tempPdf.isEncrypted: 
            try: 
                tempPdf.decrypt('') 
                print('pdf file was encrypted')
            except: 
                print("Attempted decrpytion failed") 
                
        merger = PyPDF2.PdfFileMerger()
        print("merger file created")
        
        try: 
            merger.append(tempPdf)
            print("temp file appended to merge file")
        except:
            print("No file content found") 

        try: 
            docInfoLen = len(tempPdf.documentInfo)
            print('file length found')
        except:
            print("Encrpytion Error: Couldnt Decrypt Metadata on File") 

        if docInfoLen>0: 
            try:
                merger.addMetadata(tempPdf.documentInfo)
                print('metadata written to file')
            except: 
                print("Metadata not written to file")
        temp_return = StringIO()
        merger.write(temp_return)
        return (temp_return.getvalue())

    except:
        print("There was an error retrieving the file.  Please check the URL and try again.")
        return None

def extract_text_content_and_pages(pdf):
    content = ""
    if pdf.isEncrypted: 
        try: 
            pdf.decrypt('')
        except: None
    # iterate pages
    for i in range(0, pdf.getNumPages()):
        # extract the text from each page
        try: content += pdf.getPage(i).extractText() + " \n"
        except: continue
    # collapse whitespaces
    content = " ".join(content.replace(u"\xa0", " ").split()).encode('utf-8')
    return (content,pdf.getNumPages())

# User Input:

In [33]:
user_input = {}
user_file = raw_input("Enter File Path or URL: ").lower()

if user_file[-4:] == '.pdf':
    if user_file[:4] == 'http':
        user_input.update({'URL':user_file})
        user_input.update({'File_Location':"web"})
    elif user_file[1:3] == ':\\':
        user_input.update({'Local_Path':user_file})
        user_input.update({'URL':None})
        user_input.update({'File_Location':"local"})
    else: print("File Type Error: Please enter a URL or full local path")
else: print('Only .pdf files are supported at this time')
    
if user_input['File_Location']=='web':
    existing_file = collection.find_one({u"URL":(user_input['URL']).lower()})
    if existing_file != None:
        print('The file is already in the database as UID: {}'.format(existing_file['UID']))
        print(existing_file)
    else:
        import hashlib
        file_binary=download_from_url(user_input['URL'])
        file_hash = hashlib.sha1(file_binary).hexdigest()
        print(file_hash)
        existing_file = collection.find_one({u"Hash":file_hash})
        
        import PyPDF2
        from StringIO import StringIO
        pdf_in_memory = PyPDF2.PdfFileReader(StringIO(file_binary))
        
        if existing_file != None:
            print('The file is already in the database as UID: {}'.format(existing_file['UID']))
            print(existing_file)
        else:
            print('A duplicate hash was not found in the database.')
            
            url_parts = user_input["URL"].strip().split('/') 
            user_input.update({'Domain':url_parts[2]})
            user_input.update({'Folder':url_parts[-2]})
            user_input.update({'File_Name':url_parts[-1]})
            
            user_input.update({'Hash':file_hash})

            content_extract = extract_text_content_and_pages(pdf_in_memory)
            user_input.update({'Pages':content_extract[1]})
            user_input.update({'Content':content_extract[0]})
            
            new_content_hash = hashlib.sha1(content_extract[0]).hexdigest()
            user_input.update({'Content_Hash': new_content_hash})
            
            user_input.update({'Metadata':pdf_in_memory.documentInfo})
            
            import pickle
            current_UID_position = pickle.load(open('current_uid.pickle','r'))
            user_input.update({'UID':'BSA_{}'.format(current_UID_position)})
            current_UID_position +=1
            with open('current_uid.pickle','w') as p:
                pickle.dump(current_UID_position, p)
            
            import sys
            user_input.update({'File_Size':sys.getsizeof(file_binary)})
            
            existing_content_hash_doc = collection.find_one({"Content_Hash":new_content_hash})
            
            collection.insert_one(user_input)
            
            if existing_content_hash_doc != None:
                print('A file with matching content was found as UID: {}'.format(existing_content_hash_doc['UID']))
                print(existing_content_hash_doc)
            else:
                print('This is a file with new content and has been loaded as BSA_{}'.format(current_UID_position-1))
                print('*'*75)
                print(user_input)
else:
    local_file_dict = {} #blank dictionary to add data to
    local_pdf = PyPDF2.PdfFileReader(file(user_input['Local_Path'],'rb'))
    
    local_file_dict.update({'Metadata':local_pdf.documentInfo})
    
    import os
    local_file_dict.update({'File_Size': os.path.getsize(user_input['Local_Path'])})
    
    local_file_hash = hashlib.sha1(open(user_input['Local_Path'],'rb').read()).hexdigest() 
    local_file_dict.update({'Hash':local_file_hash})
    
    path_parts = user_input['Local_Path'].strip().split('\\')
    local_file_dict.update({'File_Name':path_parts[-1]})
    
    content_extract = extract_text_content_and_pages(local_pdf)
    local_file_dict.update({'Pages':content_extract[1]})
    local_file_dict.update({'Content':content_extract[0]})
    local_file_dict.update({'Content_Hash': hashlib.sha1(content_extract[0]).hexdigest()})
    
    local_file_dict.update({'URL':None})
    local_file_dict.update({'Domain':None}) 
    local_file_dict.update({'Folder':None}) 
    
    
    #now we need to check for duplicates!
    
    bin_hash_match = collection.find_one({u"Hash":local_file_dict['Hash']})
    
    if bin_hash_match != None:
        print('The file is already in the database as UID: {}'.format(bin_hash_match['UID']))
        print(bin_hash_match)
    else:
        print('A duplicate hash was not found in the database.')
        
        content_hash_match = collection.find_one({"Content_Hash":local_file_dict['Content_Hash']})
        
        if content_hash_match != None:
            print('The file content already in the database as UID: {}'.format(content_hash_match['UID']))
            print(content_hash_match)
        else:
            print('There is no match for this file in the database. Please re-submit as a URL once the file is loaded to the web')
    print('*'*75)
    print(local_file_dict)

Enter File Path or URL: C:\Users\ajwil\Desktop\AJs PS3.pdf
{'Content': 'PROBLEM SET 3 ECON 5337 ALEXANDER WILSON 1. To complete the forecasting exercise, there are two packages/libraries that must be loaded: >library(timeSeries) >library(forecast) >library(urca) A. To load the data and plot the time series data, it requires reading the csv file into memory, converting it to a time series with the ts() function, and then plotting the time series function. >data < - read.csv("pset3.csv") >yuemp = ts(data$Unemploy, star t = c(1955,1), end = c(2015,1),freq = 4) >plot(yuemp) B. To forecast the youth unemployment rate, guesses must be made about the ARMA(p,q) structure of the model. To aid in making the guess, the autocorrelation function and partial autocorrelation fun ction should be used in order to look at any possible patterns in the data. >acf(yuemp) PROBLEM SET 3 ECON 5337 ALEXANDER WILSON We of course ignore the value at lag 0 and see multiple statistically significant, positive auto

In [3]:
# http://www.cbu.edu.zm/downloads/pdf-sample.pdf
# http://tdc-www.harvard.edu/Python.pdf
# http://www.ucs.cam.ac.uk/docs/course-notes/unix-courses/PythonAB/files/handout.pdf
# http://adventure.oa-bsa.org/files/flyers/OAHA_5_program_flyer.pdf
# http://www.scouting.org/filestore/boyscouts/pdf/Troop_Leadership_Positions.pdf
#
#
#D:\BSA_PDF_Files\BSA_107.pdf